In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
from trackml.dataset import load_event
from trackml.randomize import shuffle_hits
from trackml.score import score_event

In [3]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
%matplotlib inline

In [4]:
import os
import numpy as np
import pandas as pd
import glob
import math

In [5]:
from utils import make_uID
det = pd.read_csv('input/detectors.csv')
det.head()
hits_pd = make_uID(det)
hits_pd.shape

(18728, 4)

### load one event

In [6]:
event_prefix = 'event000001000'
hits, cells, particles, truth = load_event(os.path.join('input/train_1', event_prefix))

In [7]:
hits_with_uID = pd.merge(hits, hits_pd, on=['volume_id', 'layer_id', 'module_id'])
from utils import get_features
hits_final = get_features(hits_with_uID)
hits_final.head()

,hit_id,x,y,z,volume_id,layer_id,module_id,uID,x2,y2,z2,r2,r,eta,phi,absZ
0,1,-64.409897,-7.163700,-1502.5,7,2,1,0,-0.042829,-0.004763,-23.184208,64.807045,1503.896973,-3.837108,-3.030827,1502.5
1,2,-55.336102,0.635342,-1502.5,7,2,1,0,-0.036804,0.000423,-27.150467,55.339748,1503.518799,-3.994889,3.130112,1502.5
2,3,-83.830498,-1.143010,-1502.5,7,2,1,0,-0.055707,-0.000760,-17.921406,83.838287,1504.837158,-3.579932,-3.127959,1502.5
3,4,-96.109100,-8.241030,-1502.5,7,2,1,0,-0.063835,-0.005474,-15.576118,96.461777,1505.593262,-3.439919,-3.056055,1502.5
4,5,-62.673599,-9.371200,-1502.5,7,2,1,0,-0.041676,-0.006232,-23.709833,63.370335,1503.835815,-3.859459,-2.993168,1502.5


In [8]:
filtered_truth = truth[ (truth['weight'] > 5e-7 ) & (truth['particle_id'] != 0) ]
training = hits_final.merge(filtered_truth, on='hit_id')[['uID', 'particle_id']]

In [9]:
truth_particles = pd.Series(np.unique(training['particle_id']))
print("total particles:", truth_particles.shape[0])

total particles: 9292


### Try the RNN model

In [10]:
class HitPredictor(nn.Module):
    def __init__(self, input_dim=20, hidden_dim=20, output_dim=20,
                n_lstm_layers=1):
        super(HitPredictor, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.batch_size = 1
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_lstm_layers)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.hidden = self.init_hidden()
        self.softmax = nn.LogSoftmax(dim=1)

    def init_hidden(self):
        return (torch.zeros(self.lstm.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.lstm.num_layers, self.batch_size, self.hidden_dim))
        
    def forward(self, x):
        input_size = x.size()
        output, self.hidden = self.lstm(x, self.hidden)
        output = self.fc(output.view(len(x), -1))
        tag_scores = self.softmax(output)
        return tag_scores

In [ ]:
class HitPredictor(nn.Module):
    def __init__(self, input_dim=20, hidden_dim=20, output_dim=20,
                n_lstm_layers=1):
        super(HitPredictor, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.batch_size = 1
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_lstm_layers)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.hidden = self.init_hidden()
        self.softmax = nn.LogSoftmax(dim=1)

    def init_hidden(self):
        return (torch.zeros(self.lstm.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.lstm.num_layers, self.batch_size, self.hidden_dim))
        
    def forward(self, x):
        input_size = x.size()
        output, self.hidden = self.lstm(x, self.hidden)
        output = self.fc(output.view(len(x), -1))
        tag_scores = self.softmax(output)
        return tag_scores

In [11]:
total_modules = 18728

In [12]:
def inputHits(series):
    tensor = torch.zeros(len(series), 1, total_modules+1)
    for idx, h in enumerate(series):
        tensor[idx][0][h] = 1

    return tensor

In [13]:
def targetHits(series):
    module_idx = series[1:].tolist()
    module_idx.append(total_modules)
    # module_idx += [total_modules]*(21 - len(series))
    return torch.LongTensor(module_idx)

In [14]:
def randomTrack():
    pID = truth_particles.sample(1).values
    hit_series = training[training['particle_id'] == pID[0]]['uID'].values
    input_tensor = inputHits(hit_series)
    target_tensor = targetHits(hit_series)
    return input_tensor, target_tensor

In [15]:
def getInputTarget(hit_series):
    input_tensor = inputHits(hit_series)
    target_tensor = targetHits(hit_series)
    return input_tensor, target_tensor

In [46]:
input2_, target2_ = randomTrack()
print(input2_.size())
print(len(input2_))
print(target2_.size())

torch.Size([12, 1, 18729])
12
torch.Size([12])


In [17]:
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [18]:
def check_result(input_, tags_, target_):
    with torch.no_grad():
        print("inputs:", torch.reshape(torch.argmax(input_, dim=2), (-1,)))
        print("predictions:", torch.argmax(tags_, dim=1))
        print("target:", target_)

### Train RNN with all tracks in this event

In [39]:
n_iters = truth_particles.shape[0]

n_iters = 100
print_every = n_iters/10
plot_every = 200
all_losses = []
total_loss = 0
start = time.time()


criterion = nn.NLLLoss()
rnn = HitPredictor(input_dim=total_modules+1, hidden_dim=total_modules+1, 
                   output_dim=total_modules+1,
                   n_lstm_layers=2)
optimizer = optim.SGD(rnn.parameters(), lr=0.01)

for iter_ in range(n_iters):
    pID = truth_particles.values[iter_]

    # prepare the inputs
    input_, target_ = getInputTarget(training[training['particle_id'] == pID]['uID'].values)
    
    # training
    rnn.zero_grad()
    rnn.hidden = rnn.init_hidden()
    tag_scores = rnn(input_)
    
    loss = criterion(tag_scores, target_)
    loss.backward()
    optimizer.step()
    
    total_loss += loss
    if iter_ % print_every == 0:
        print('%s (%d %d%%) %.4f' % (timeSince(start), iter_, iter_ / n_iters * 100, loss))
        check_result(input_, tag_scores, target_)
        
    if iter_ % plot_every == 0:
        all_losses.append(total_loss / plot_every)
        total_loss = 0

print("I am done")

3m 44s (0 0%) 9.8405
inputs: tensor([   864,   1197,   1729,   2608,   5376,   6358,   7652,  15401])
predictions: tensor([  1629,   7613,   7613,   7613,   7613,  16516,   1629,   1629])
target: tensor([  1197,   1729,   2608,   5376,   6358,   7652,  15401,  18728])
45m 34s (10 10%) 9.8344
inputs: tensor([   77,   185,   292,   400,   508,   616,   618,   724,   726,
         4122])
predictions: tensor([ 12334,   1791,   1791,   1791,   1791,   1789,   1789,   1789,
          1789,   1789])
target: tensor([   185,    292,    400,    508,    616,    618,    724,    726,
          4122,  18728])
85m 38s (20 20%) 9.8351
inputs: tensor([   853,   1175,   1693,   2475,   5347,   6261,   7514,   9196,
         12797,  15262])
predictions: tensor([ 18728,  18728,  18728,  18728,  18728,  18728,  18728,  18728,
         18728,  18728])
target: tensor([  1175,   1693,   2475,   5347,   6261,   7514,   9196,  12797,
         15262,  18728])
129m 8s (30 30%) 9.8369
inputs: tensor([   854,   117

In [42]:
max_nhits = 20
def check_training(start_hit):
    with torch.no_grad():
        input_tensor = inputHits([start_hit])
        print(input_tensor.size())
        print(input_tensor[0][0][start_hit])
        rnn.hidden = rnn.init_hidden()
        out_hits = []
        for i in range(max_nhits):
            output = rnn(input_tensor)
            topv, topi = output.topk(1)
            topi = topi[0][0]
            if topi == total_modules:
                print("Hit the last hits")
                break
            else:
                out_hits.append(topi)
            input_tensor = inputHits([topi])
        return out_hits

In [43]:
check_training(3456)

torch.Size([1, 1, 18729])
tensor(1.)
Hit the last hits


[]

In [38]:
torch.save(rnn.state_dict(), "trained_1event_rnn_model")

In [ ]:
input_, target = randomTrack()
rnn = HitPredictor(input_dim=total_modules+1, hidden_dim=512, output_dim=total_modules+1)
n_iters = 100
print_every = 10
all_losses = []
total_loss = 0

#target.unsqueeze_(-1)
rnn.hidden = rnn.init_hidden()

rnn.zero_grad()
loss = 0
criterion = nn.NLLLoss()

print("input size:", input_.size())
print('target size:', target.size())
output = rnn(input_)
print("output size:", output.size())
print(target)
loss = criterion(output, target)
print(loss)

In [57]:
rnn2 = nn.LSTM(6, 20)
input = torch.randn(1)
print(input.size())
print(input.type())
print(input.dtype)
h0 = torch.randn(2, 1, 20)
c0 = torch.randn(2, 1, 20)
output, hn = rnn2(input, (h0, c0))
print(output)
print(output.size())
print(hn)
print(hn[0].size())

torch.Size([1])
torch.FloatTensor
torch.float32


RuntimeError: dimension out of range (expected to be in range of [-1, 0], but got 1)

In [163]:
m = nn.LogSoftmax(dim=1)
loss = nn.NLLLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.tensor([1, 0, 4])
print(target.size())
print(m(input).size())
output = loss(m(input), target)

In [61]:
embedding = nn.Embedding(10, 3)
input = torch.LongTensor([1,2,4,5])
print(input)
embeds = embedding(input)

tensor([ 1,  2,  4,  5])


In [62]:
print(embeds.size())
print(input.size())
print(len(input))

torch.Size([4, 3])
torch.Size([4])
4


In [63]:
print(embeds)
new_embeds = embeds.view(len(input), 1, -1)
print(new_embeds)
print(new_embeds.size())

tensor([[-0.5956, -0.3846,  0.6718],
        [ 0.4788,  0.3929,  0.4087],
        [ 0.3090,  0.4232,  1.3487],
        [ 2.1231, -0.9157, -1.3121]])
tensor([[[-0.5956, -0.3846,  0.6718]],

        [[ 0.4788,  0.3929,  0.4087]],

        [[ 0.3090,  0.4232,  1.3487]],

        [[ 2.1231, -0.9157, -1.3121]]])
torch.Size([4, 1, 3])
